In [2]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

import sys

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'
#capabilities=firefox_capabilities,
firefox = webdriver.Firefox(executable_path=geckoPath)
#driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')
driver = webdriver.Firefox( executable_path=geckoPath)
try:
    firefox.get('https://www.crunchbase.com/organization/apple')
except:
    pass
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(2)
driver.set_page_load_timeout(30)

def existence_in_crunchbase(name):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+name
    url_crunchbase = None
    try:
        driver.get(url)
    except KeyboardInterrupt:
        sys.exit()
    except:
        print('timeout of phantomjs')
        pass
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url not in url_crunchbase:
            url_crunchbase = None
    
    return url_crunchbase
            

def extract_company_techcrunch(name,url):
    try:
        firefox.get(url)
    except:
        pass
    founded = None
    employees = None
    company_name = ''
    blocks_dt = []
    blocks_dd = []
    try:
        soup = BeautifulSoup(firefox.page_source, "html.parser")
        blocks_dt = soup.select("div.details dt")
        blocks_dd = soup.select("div.details dd")
        company_name_tag = soup.select_one("#profile_header_heading")
        if company_name_tag == None:
            company_name = ''
        else:
            company_name = company_name_tag.getText()
    except:
        pass

    i = 0
    while company_name.lower() != name.lower() or len(blocks_dt)==0 :
        time.sleep(0.3)
        try:
            soup = BeautifulSoup(firefox.page_source, "html.parser")
            company_name_tag = soup.select_one("#profile_header_heading")
            if company_name_tag:
                company_name = company_name_tag.getText()
            else:
                company_name = ''
            blocks_dt = soup.select("div.details dt")
            blocks_dd = soup.select("div.details dd")
        except KeyboardInterrupt:
            sys.exit()
        except:
            pass
        i=i+1
        if i >= 100:
            print('fail to crawl ', name , ' in crunchbase')
            break

    
    if company_name.lower() == name.lower():
        for index,block in enumerate(blocks_dt):
            if 'Founded' in  block.getText() : 
                founded = blocks_dd[index].getText()
            if 'Employees' in block.getText():
                employees = blocks_dd[index].getText().split('|')[0]
    print( name , founded , employees)
    return founded,employees



In [5]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
            search_label = "0"
            if "search_label" in company:
                search_label = company["search_label"]
                
            if search_label=="0" and index >2750:
                url_crunchbase = existence_in_crunchbase(name)
                
                if url_crunchbase != None:
                    print(name, ' in crunchbase')
                    i=i+1
                    founded,employees = extract_company_techcrunch(name,url_crunchbase)
                    company["search_label"] = url_crunchbase
                    if founded is not None:
                        company["foundationDate"] = founded
                    if employees is not None:
                        company["number_of_employees"] = employees
                    
                else:
                    company["search_label"] = str(0)
                    print(name, 'not in crunchbase')
            else:
                print("skip: ", name)
            
            if (index+1)%100 == 0:
                self.save_to_disk()
        self.save_to_disk()            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [6]:
CM = CompaniesManager()
#CM.extract_companies()
CM.extend_crunch()


skip:  SENSORO
skip:  Microsoft Corp
skip:  Sumitomo
skip:  Manchester City Verve
skip:  SalesWings
skip:  SA tech
skip:  Avito
skip:  Tshimologong Innovation Hub
skip:  Flickr
skip:  Niantic Labs
skip:  Oblong
skip:  Osterhout Design Group
skip:  Niantic
skip:  Oblong Industries
skip:  Mezzanine
skip:  The Pokemon Company
skip:  GamesBeat
skip:  BNJ Marketing
skip:  DormaKaba
skip:  Berkshire companies
skip:  Bezos
skip:  Private Equity
skip:  Utrip
skip:  Greylock
skip:  Elsight
skip:  LiveU
skip:  WAN
skip:  Vodaphone
skip:  Optus
skip:  GoRemote
skip:  Agri Alliance
skip:  Bitbond
skip:  Obotritia
skip:  FiveAI
skip:  GuestReady
skip:  Carspring
skip:  Moneybox
skip:  Drivetribe
skip:  Realla
skip:  Habito
skip:  Sling & Stone
skip:  Crowdcube
skip:  Cali Rice
skip:  Nominet Trust
skip:  3D Robotics
skip:  AgFunder
skip:  Airware
skip:  Arbe Robotics
skip:  Zipline International
skip:  Redbird
skip:  Bloomberg Businessweek
skip:  Yuneec Electric Aviation
skip:  CyPhy Works
skip:  E

skip:  SA Ignite
skip:  TRECC
skip:  Hublot
skip:  PayU
skip:  Visa Nest
skip:  Rural Broadband
skip:  Gram Marg Solutions
skip:  Chivas Regal
skip:  Mzansi Magic
skip:  Thumbzup Innovation
skip:  AfricInvest
skip:  Bpifrance
skip:  Societe Generale
skip:  NSBC
skip:  MTN Channel
skip:  Magento
skip:  Entersekt
skip:  Arbour Group
skip:  DFS Lab
skip:  iHub
skip:  TFG
skip:  Social Lender
skip:  XL Embassy Travel
skip:  Holiday Factory
skip:  The Singer Group
skip:  Beachcomber
skip:  Tagmarshal
skip:  Phillips Consulting Group
skip:  AIS Media
skip:  Numeris Media Group
skip:  Zapphaire Events
skip:  Angel Investors
skip:  Angel
skip:  Dropbox
skip:  Reddit
skip:  Amref Health Africa
skip:  WePay
skip:  AgroCenta
skip:  Oasis 500
skip:  Sanad
skip:  IDWork
skip:  Interswitch
skip:  Investment Okechukwu Enelamah
skip:  Andela
skip:  Onuegbu
skip:  Potraz
skip:  TechZim
skip:  CCDI
skip:  Disa Vascular
skip:  Incitech
skip:  Finfind
skip:  2U
skip:  DiGame
skip:  Olivine Technology
skip

skip:  Studypool
skip:  Startup Change Group
skip:  Peergrade
skip:  Vlada Lotkina
skip:  JotForm
skip:  Milo Sensors
skip:  SAM Labs
skip:  Aston Martin
skip:  ThirdLove
skip:  Hipchat
skip:  Josh.ai
skip:  Faraday
skip:  Hong Bae
skip:  Vive
skip:  Viveport
skip:  Time Inc
skip:  Dreamworks
skip:  Weta Digital.
skip:  AnchorFree
skip:  Gorodyansky
skip:  Gartner, Inc.
skip:  Nest Labs
skip:  Kellogg Scho
skip:  Moo.do
skip:  Firebase
skip:  bgC3
skip:  Nutonomy
skip:  Nordstrom Rack
skip:  NPD Group
skip:  Macy
skip:  Deloitte Consulting LLP
skip:  Wet Seal
skip:  Payless
skip:  The Limited
skip:  car2go
skip:  Moovel Group
skip:  Ars Technica
skip:  Revolv
skip:  Propelify Innovation
skip:  Thrive Global
skip:  The Boring Company
skip:  Federal Communications
skip:  DC
skip:  Oculus Research
skip:  Zipkin
skip:  Udacity
skip:  Chandrasekaran
skip:  Snapdeal
skip:  TaskRabbit
skip:  ADP
skip:  USA Today
skip:  Gusto
skip:  Fidelity
skip:  Money Intelligence
skip:  Raymond James Finan

skip:  NEW INC
skip:  Match.com
skip:  Crelate
skip:  Ascentium
skip:  Avanade
skip:  Puzzazz
skip:  Heptio
skip:  Nohla Therapeutics
skip:  Arivale
skip:  InfoSpace
skip:  Snowflake
skip:  Snowflake Computing
skip:  Azure Data Warehouse
skip:  GigaOm
skip:  Qumulo
skip:  Isilon Systems
skip:  Qumulo Core
skip:  Tyche Partners
skip:  Sinclair Oil
skip:  Ripl
skip:  RealNetworks
skip:  Smilebox
skip:  Kauffman
skip:  EveryMove
skip:  higi
skip:  High Fidelity
skip:  Limitless
skip:  Bungie
skip:  ToyTalk
skip:  GrubHub
skip:  Intelius
skip:  Dwellable
skip:  Versium
skip:  TalentWise
skip:  Legal Services Corporation
skip:  MetaJure
skip:  Georgia Legal Services
skip:  MassChallenge
skip:  Tellus Labs
skip:  Bain & Company
skip:  Adhesys Medical
skip:  Socedo
skip:  Extreme Networks
skip:  HubSpot
skip:  9Mile Labs
skip:  SheerID
skip:  Ivy Softworks
skip:  CuePath Innovation
skip:  CuePath
skip:  DataChannel
skip:  XMLfund
skip:  Aventail
skip:  Compuserve
skip:  GlobeScape
skip:  uZoo

skip:  MilkFlow
skip:  The Gift Concierge Co
skip:  Employment Hero
skip:  Loop11
skip:  Sendle
skip:  Genesis Energy
skip:  Creative HQ
skip:  Callaghan Innovation
skip:  Zencastr
skip:  Auphonic
skip:  GetSwift
skip:  CIMIC Limited
skip:  QSRH
skip:  ACMI
skip:  Runway Geelong
skip:  Deakin
skip:  TechSydney
skip:  Assi
skip:  CSIRO Innovation Fund
skip:  Doshii
skip:  Impos
skip:  NSW Innovation Concierge
skip:  Primary Industries
skip:  SmartBooks Online
skip:  GlamCrew
skip:  GlamX
skip:  Brickx
skip:  CoHome
skip:  REA Group
skip:  Core Logic
skip:  STEM Club
skip:  TwoPeas
skip:  eHarmony
skip:  Postr
skip:  Boost Mobile
skip:  Skinny Mobile
skip:  Lebara
skip:  Rision
skip:  GreenSync
skip:  Clean Energy Finance Corporation
skip:  United Energy
skip:  CEFC
skip:  SCVP
skip:  Jugglr
skip:  Local Measure
skip:  AccorHotels
skip:  Club Med
skip:  Bugcrowd
skip:  Panama Papers
skip:  Alpaca Maps
skip:  Vithoulkas
skip:  Vivo Cafe
skip:  Eagle Startups
skip:  Eagle Waves Radio
skip:

ACE & Company not in crunchbase
skip:  Pacific Century Group
henQ not in crunchbase
skip:  ProGlove
skip:  EIT Digital
UnternehmerTUM not in crunchbase
Bayern Kapital not in crunchbase
skip:  Gettylab
Autoglass not in crunchbase
skip:  SPARTAN
skip:  OptoForce
skip:  ATI Industrial Automation
skip:  FloraQueen
FloraQuen not in crunchbase
skip:  Swiss Post
DeliveryHero not in crunchbase
skip:  Taxibeat
Adecco Group not in crunchbase
Royal DSM not in crunchbase
Boyan not in crunchbase
skip:  Adaptive Simulations
skip:  Schraubenfabrik
Das kommod not in crunchbase
skip:  Betahaus
Stockwerk not in crunchbase
Yurp not in crunchbase
skip:  CallPage
skip:  Neurescue
Rimac not in crunchbase
skip:  Fresh Check
skip:  PaperHive
Srsen not in crunchbase
Sabadell Asset Management not in crunchbase
Nexity not in crunchbase
skip:  Aruba Cloud
skip:  the Cloud
skip:  Teamleader
skip:  NACUE
skip:  presono
HMG Invest not in crunchbase
Greiner Packaging International not in crunchbase
HAUSER Kuhlmobel n

FanDuel  in crunchbase
FanDuel June, 2007 13 in Crunchbase
Trail Answers  in crunchbase
Trail Answers None None
eBay India  in crunchbase
eBay India Unknown 3 in Crunchbase
Jabong not in crunchbase
Wright Electric not in crunchbase
MacRebur  in crunchbase
MacRebur 2016 1 in Crunchbase
Ola  in crunchbase
Ola December 3, 2010 1k - 5k 
Howz  in crunchbase
Howz Unknown None found in Crunchbase
EDF Energy  in crunchbase
EDF Energy 1990 10k + 
Blue Lab  in crunchbase
Blue Lab Unknown None
C. Chocolate not in crunchbase
Tiger Global not in crunchbase
Alfamart not in crunchbase
P2i  in crunchbase
P2i 2004 51 - 100 
Bango  in crunchbase
Bango October 1, 1999 51 - 100 
Macco Robotics Macco Robotics not in crunchbase
Clik not in crunchbase
Mobovivo  in crunchbase
Mobovivo April 1, 2009 11 - 50 
Familyar not in crunchbase
Familiyar not in crunchbase
SafeToNet  in crunchbase
SafeToNet 2016 11 - 50 
Weidman Powers not in crunchbase
Project Include  in crunchbase
Project Include May 1, 2016 5 in Crun

Tovala  in crunchbase
Tovala May, 2015 1 - 10 
Veggie Grill  in crunchbase
Veggie Grill 2006 501 - 1k 
Petit Organics not in crunchbase
Alpha  in crunchbase
Alpha 2014 11 - 50 
Skybuds not in crunchbase
Eversign  in crunchbase
fail to crawl  Eversign  in crunchbase
Eversign None None
Saul Ewing LLP  in crunchbase
Saul Ewing LLP Unknown None found in Crunchbase
Tropicana not in crunchbase
Opus Virtual Offices  in crunchbase
Opus Virtual Offices Unknown None found in Crunchbase
Fox Sports  in crunchbase
Fox Sports 1994 251 - 500 
MTI  in crunchbase
MTI 1983 None
CorrectDeck not in crunchbase
Elev8 Consulting Group Building not in crunchbase
AKTA not in crunchbase
MetaPipe not in crunchbase
Bryght not in crunchbase
ELLE Decor  in crunchbase
ELLE Decor Unknown 1 - 10 
Chapman & Company not in crunchbase
Chase Pay  in crunchbase
fail to crawl  Chase Pay  in crunchbase
Chase Pay None None
LevelUp  in crunchbase
LevelUp 2008 51 - 100 
Cosmic Watch not in crunchbase
StoryTerrace not in crunchb

Wallarm  in crunchbase


KeyboardInterrupt: 